<a href="https://colab.research.google.com/github/babypotatotang/PolicySuggestion-through-RevitalizationUnusedSchool/blob/main/Data%20Crawling/%20Crawling_StudentsNum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

from selenium.webdriver.support.select import Select

from time import sleep
import csv

In [ ]:
def write_csv(name,num,adr):
    with open('element.csv','a',newline='') as f:
        field=[]
        wr = csv.writer(f)
        
        field.append(name)
        field.append(num)
        field.append(adr)
        
        wr.writerow(field)

In [ ]:
fields=['학교이름','학생수','주소']

with open('element.csv','a',newline='') as f:
    wr = csv.writer(f)
    wr.writerow(fields)

url='http://eduinfo.go.kr/portal/theme/schNmlStatusPage.do'

options=webdriver.ChromeOptions()
options.add_argument("--start-maximized")

browser=webdriver.Chrome('chromedriver',options=options)
browser.get(url) #해당 브라우저 열기
browser.execute_script("window.scrollTo(0, 500)")

#경남 선택
select=Select(browser.find_element_by_id("eduOffcDivCd"))
select.select_by_value("S10") 

#학교 선택 
select=Select(browser.find_element_by_id("schlKndCd"))
select.select_by_value("H") #고등
num=191
idx=1
print(type(num))

#검색 클릭
search=browser.find_element_by_xpath('//*[@id="search_btn"]').click()
sleep(8)

In [ ]:
while idx < num:
    idx2=idx

    list=browser.find_element_by_xpath('//*[@id="schList"]/li[{}]/a/em'.format(idx2))
    list.click()
    
    school_name=list.text #학교 이름

    browser.execute_script("window.scrollTo(0, 1100)")
    
    tr=browser.find_element_by_xpath("//*[@id='tableSchulRdnma']")
    address=tr.text
    print(address)
        
    browser.execute_script("window.scrollTo(0, 1900)")
    sleep(5)

    tr=browser.find_element_by_xpath("//*[@id='sheetTable']/tbody/tr[2]")
    td = tr.find_elements_by_tag_name("td")
    student_num = "{}\n".format(td[8].text) #학생 수
    
    print("{}, {}, {}\n".format(school_name,student_num,address))   
    write_csv(school_name,student_num,address)
    print("{}건 입력 완료".format(idx2))
    
    idx=idx+1
    sleep(5)
    browser.execute_script("window.scrollTo(0, 500)")